In [ ]:
import numpy as np 
import pandas as pd 
import os
import warnings
warnings.filterwarnings('ignore')

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn import preprocessing

In [ ]:
def fillextsources(label, features, app_train, app_test):
    app_train_with_value = app_train[app_train[label].notnull()] #1
    app_test_with_value = app_test[app_test[label].notnull()]    #3
    app_train_no_value = app_train[app_train[label].isnull()]    #2
    app_test_no_value = app_test[app_test[label].isnull()]       #4
    len1 = len(app_train_with_value)
    len3 = len(app_test_with_value)
    len2 = len(app_train_no_value)
    len4 = len(app_test_no_value)
    # A = 1 + 3
    A = pd.concat([app_train_with_value, app_test_with_value], axis=0)
    # B = 2 + 4
    B = pd.concat([app_train_no_value, app_test_no_value], axis=0)
    
    # A
    # fill median for each column
    index = 0
    for feature in features:
        A[feature].fillna(meds[++index])
    
    
    # B
    # fill median for each column
    index = 0
    for feature in features:
        B[feature].fillna(meds[++index])
        
    # x_train y_train  x_test
    x_train = A[features]
    y_train = A[label]
    x_test = B[features]
    
    # model train and predict
    model = linear_model.LinearRegression()
    model.fit(x_train, y_train)
    y_test = model.predict(x_test) # filled EXT_SOURCE_1
    
    # fill B
    B[label] = y_test
    
    app_train_new = pd.concat([A[0:len1], B[0:len2]], axis=0)
    app_test_new = pd.concat([A[len1:len1+len3], B[len2:len2+len4]], axis=0)
    app_test_new.sort_values(by=['SK_ID_CURR'])
    
    return app_train_new, app_test_new
    
 
    

In [ ]:
labels = ['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']
app_train = pd.read_csv('application_train.csv')
app_test = pd.read_csv('application_test.csv')


features = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_ID_PUBLISH', 'DAYS_REGISTRATION',  'AMT_CREDIT', 'AMT_INCOME_TOTAL'
           , 'REGION_POPULATION_RELATIVE']

preprocessing.scale(app_train[features])
preprocessing.scale(app_test[features])

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243

# Replace the anomalous values with nan
app_train['DAYS_EMPLOYED'].replace({365243: -1224}, inplace = True)

#app_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram');
#plt.xlabel('Days Employment');

app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: -1224}, inplace = True)



#print('There are %d anomalies in the test data out of %d entries' % (app_test["DAYS_EMPLOYED_ANOM"].sum(), len(app_test)))

app_train.loc[:,'DAYS_BIRTH'] /= -365
app_test.loc[:,'DAYS_BIRTH'] /= -365

meds = []
for feature in features:
    ds = app_train[feature].append(app_test[feature]) 
    meds.append(ds.median())

for label in labels:
    app_train_new, app_test_new = fillextsources(label, features, app_train, app_test)
    app_train = app_train_new
    app_test = app_test_new



In [ ]:
# model logistc regression
x_train_final = app_train[labels]
y_train_final = app_train['TARGET']
x_test_final = app_test[labels]

model = LogisticRegression()
model.fit(x_train_final, y_train_final)
prediction_proba = model.predict_proba(x_test_final)[:, 1]




In [ ]:
# Submission dataframe
submit = app_test[['SK_ID_CURR']]
submit['TARGET'] = prediction_proba

# convert to csv
submit.to_csv('LR.csv', index = False)